# Demo 3: Embeddings and Semantic Search

In this demo, you'll learn how embeddings transform text into numbers and enable semantic search! We'll explore:

1. **What are embeddings?** - Converting text to vectors
2. **Creating embeddings** - Using OpenAI's API
3. **Similarity calculations** - Finding related content
4. **Semantic search** - Finding reviews by meaning, not just keywords
5. **Clustering** - Grouping similar items automatically

## Learning Objectives

By the end of this demo, you'll understand:
- How embeddings capture semantic meaning in numbers
- How to measure similarity between texts
- How semantic search differs from keyword search
- How to build a simple recommendation system

## Setup

First, let's import the required libraries.

In [ ]:
# Install required packages if needed
# !pip install openai numpy scikit-learn matplotlib pandas

from openai import OpenAI
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd
import os

# Initialize OpenAI client
api_key = os.getenv("OPENAI_API_KEY", "your-api-key-here")
client = OpenAI(
    api_key=api_key,
    base_url="https://openai.vocareum.com/v1" if api_key.startswith("voc") else None
)

print("✅ Libraries imported successfully!")

## Part 1: What Are Embeddings?

Embeddings are **numerical representations of text** that capture semantic meaning.

Think of it this way:
- Text: "The movie was amazing!" 
- Embedding: [0.234, -0.891, 0.456, ...] (1536 numbers for OpenAI's model)

Similar meanings = similar numbers!

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    """Get embedding vector for a text."""
    text = text.replace("\n", " ")  # Clean the text
    response = client.embeddings.create(
        input=[text],
        model=model
    )
    return response.data[0].embedding

# Get embeddings for sample texts
text1 = "I love this product!"
text2 = "This is terrible quality."
text3 = "Great purchase, very happy!"

emb1 = get_embedding(text1)
emb2 = get_embedding(text2)
emb3 = get_embedding(text3)

print("📊 Embedding Details:\n")
print(f"Text: '{text1}'")
print(f"Embedding length: {len(emb1)} dimensions")
print(f"First 10 values: {emb1[:10]}")
print(f"\nThese {len(emb1)} numbers capture the meaning of the text!")

### 💡 Key Insight

Embeddings are high-dimensional vectors (1536 dimensions!) where:
- Each dimension captures some aspect of meaning
- Similar texts have similar embedding vectors
- You can do math with meanings! ("king" - "man" + "woman" ≈ "queen")

## Part 2: Calculating Cosine Similarity

**Cosine similarity** measures how similar two embeddings are.

- 1.0 = identical
- 0.9+ = very similar
- 0.7-0.9 = somewhat similar
- < 0.5 = not very similar

In [ ]:
def calculate_similarity(emb1, emb2):
    """Calculate cosine similarity between two embeddings."""
    emb1 = np.array(emb1).reshape(1, -1)
    emb2 = np.array(emb2).reshape(1, -1)
    return cosine_similarity(emb1, emb2)[0][0]

# Compare our three texts
sim_1_2 = calculate_similarity(emb1, emb2)
sim_1_3 = calculate_similarity(emb1, emb3)
sim_2_3 = calculate_similarity(emb2, emb3)

print("🔍 Similarity Comparison:\n")
print(f"'{text1}' vs '{text2}': {sim_1_2:.4f} (different sentiment)")
print(f"'{text1}' vs '{text3}': {sim_1_3:.4f} (similar sentiment)")
print(f"'{text2}' vs '{text3}': {sim_2_3:.4f} (opposite sentiment)")

print("\n💡 Notice how positive reviews are more similar to each other!")

## Part 3: Sample Product Reviews Dataset

Let's create a realistic dataset of product reviews to work with.

In [ ]:
# Sample product reviews
reviews = [
    {"id": 1, "text": "This backpack is incredibly durable and has lots of storage space. Perfect for hiking!", "category": "positive"},
    {"id": 2, "text": "The water bottle leaks constantly. Very disappointed with the quality.", "category": "negative"},
    {"id": 3, "text": "Amazing quality! The material is waterproof and the zippers are sturdy. Highly recommend.", "category": "positive"},
    {"id": 4, "text": "Terrible customer service. They never responded to my emails about the defective product.", "category": "negative"},
    {"id": 5, "text": "Great value for money. The backpack exceeded my expectations for the price.", "category": "positive"},
    {"id": 6, "text": "The product arrived damaged and the return process was a nightmare.", "category": "negative"},
    {"id": 7, "text": "Love the design and color options. The bag is stylish and functional.", "category": "positive"},
    {"id": 8, "text": "Not worth the money. Cheap materials that tore after one week of use.", "category": "negative"},
    {"id": 9, "text": "Excellent for outdoor adventures! Kept all my gear dry during a rainstorm.", "category": "positive"},
    {"id": 10, "text": "The straps are uncomfortable and dig into my shoulders. Poor ergonomic design.", "category": "negative"},
    {"id": 11, "text": "Perfect size for day trips. Fits my laptop, water bottle, and snacks comfortably.", "category": "positive"},
    {"id": 12, "text": "Zipper broke after two uses. Don't waste your money on this.", "category": "negative"},
    {"id": 13, "text": "Fast shipping and great packaging. The product quality is outstanding!", "category": "positive"},
    {"id": 14, "text": "Way too expensive for what you get. Similar products cost half the price.", "category": "negative"},
    {"id": 15, "text": "This is my third purchase from this brand. Always reliable and high quality.", "category": "positive"}
]

# Display reviews
df = pd.DataFrame(reviews)
print("📝 Product Reviews Dataset:\n")
print(df.to_string(index=False))
print(f"\nTotal reviews: {len(reviews)}")
print(f"Positive: {sum(1 for r in reviews if r['category'] == 'positive')}")
print(f"Negative: {sum(1 for r in reviews if r['category'] == 'negative')}")

## Part 4: Creating Embeddings for All Reviews

Let's create embeddings for our entire dataset.

In [ ]:
print("🔄 Creating embeddings for all reviews...\n")

# Get embeddings for all reviews
for review in reviews:
    review['embedding'] = get_embedding(review['text'])
    print(f"✅ Review {review['id']}: Embedded")

print(f"\n✅ Created {len(reviews)} embeddings!")
print(f"Each embedding has {len(reviews[0]['embedding'])} dimensions")

## Part 5: Semantic Search - Finding Similar Reviews

Now let's search for reviews by **meaning**, not just keywords!

In [ ]:
def semantic_search(query, reviews, top_k=3):
    """Find the most similar reviews to a query."""
    # Get query embedding
    query_embedding = get_embedding(query)
    
    # Calculate similarity with all reviews
    for review in reviews:
        review['similarity'] = calculate_similarity(query_embedding, review['embedding'])
    
    # Sort by similarity
    sorted_reviews = sorted(reviews, key=lambda x: x['similarity'], reverse=True)
    
    return sorted_reviews[:top_k]

# Example searches
queries = [
    "Looking for a sturdy bag for outdoor activities",
    "Had issues with product breaking quickly",
    "Good deal for the price"
]

for query in queries:
    print(f"🔍 Query: '{query}'\n")
    results = semantic_search(query, reviews, top_k=3)
    
    for i, result in enumerate(results, 1):
        print(f"  {i}. [Similarity: {result['similarity']:.4f}] {result['text']}")
    
    print("\n" + "="*80 + "\n")

### 💡 Key Insight

Notice how semantic search finds relevant reviews even when:
- The exact words don't match
- Synonyms are used ("sturdy" → "durable")
- The concept is expressed differently

This is **much more powerful** than keyword search!

## Part 6: Visualizing Embeddings with PCA

Let's reduce 1536 dimensions down to 2D so we can visualize the embeddings!

In [ ]:
# Extract embeddings as a matrix
embeddings_matrix = np.array([r['embedding'] for r in reviews])

# Reduce to 2D using PCA
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings_matrix)

# Separate by category
positive_indices = [i for i, r in enumerate(reviews) if r['category'] == 'positive']
negative_indices = [i for i, r in enumerate(reviews) if r['category'] == 'negative']

# Plot
plt.figure(figsize=(12, 8))

plt.scatter(
    embeddings_2d[positive_indices, 0],
    embeddings_2d[positive_indices, 1],
    c='green',
    marker='o',
    s=200,
    alpha=0.6,
    edgecolors='darkgreen',
    linewidths=2,
    label='Positive Reviews'
)

plt.scatter(
    embeddings_2d[negative_indices, 0],
    embeddings_2d[negative_indices, 1],
    c='red',
    marker='s',
    s=200,
    alpha=0.6,
    edgecolors='darkred',
    linewidths=2,
    label='Negative Reviews'
)

# Add labels for each point
for i, review in enumerate(reviews):
    plt.annotate(
        f"#{review['id']}",
        (embeddings_2d[i, 0], embeddings_2d[i, 1]),
        fontsize=9,
        ha='center',
        va='center'
    )

plt.xlabel('First Principal Component', fontsize=12)
plt.ylabel('Second Principal Component', fontsize=12)
plt.title('Product Reviews - Embedding Visualization (PCA)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 Notice how positive and negative reviews cluster together!")
print("Similar sentiment = similar location in the embedding space.")

## Part 7: Clustering Reviews by Theme

Let's use K-Means clustering to automatically group reviews by topic!

In [ ]:
# Perform clustering
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(embeddings_matrix)

# Add cluster labels to reviews
for i, review in enumerate(reviews):
    review['cluster'] = cluster_labels[i]

# Show clusters
print(f"🎯 Automatic Clustering into {n_clusters} Groups:\n")

for cluster_id in range(n_clusters):
    cluster_reviews = [r for r in reviews if r['cluster'] == cluster_id]
    print(f"\n📦 Cluster {cluster_id + 1} ({len(cluster_reviews)} reviews):")
    print("="*80)
    
    for review in cluster_reviews:
        print(f"  [{review['category'].upper()}] {review['text']}")

# Visualize clusters
plt.figure(figsize=(12, 8))

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
for cluster_id in range(n_clusters):
    cluster_indices = [i for i, r in enumerate(reviews) if r['cluster'] == cluster_id]
    plt.scatter(
        embeddings_2d[cluster_indices, 0],
        embeddings_2d[cluster_indices, 1],
        c=colors[cluster_id],
        marker='o',
        s=200,
        alpha=0.6,
        edgecolors='black',
        linewidths=2,
        label=f'Cluster {cluster_id + 1}'
    )
    
    # Add labels
    for i in cluster_indices:
        plt.annotate(
            f"#{reviews[i]['id']}",
            (embeddings_2d[i, 0], embeddings_2d[i, 1]),
            fontsize=9,
            ha='center',
            va='center',
            fontweight='bold'
        )

plt.xlabel('First Principal Component', fontsize=12)
plt.ylabel('Second Principal Component', fontsize=12)
plt.title('Product Reviews - Clustered by Similarity', fontsize=14, fontweight='bold')
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Clusters might represent themes like: quality issues, customer service, product features, etc.")

## Part 8: Practical Applications

Let's explore real-world uses of embeddings!

In [ ]:
print("🚀 Practical Applications of Embeddings:\n")

# Application 1: FAQ Matching
print("1️⃣ FAQ Matching - Find relevant help articles")
print("="*80)

faq_question = "How do I return a damaged item?"
related_reviews = semantic_search(faq_question, reviews, top_k=2)

print(f"Customer question: '{faq_question}'\n")
print("Related customer experiences:")
for i, review in enumerate(related_reviews, 1):
    print(f"  {i}. {review['text']}")

print("\n" + "="*80 + "\n")

# Application 2: Duplicate Detection
print("2️⃣ Duplicate Detection - Find similar/duplicate reviews")
print("="*80)

# Pick a review and find most similar ones
target_review = reviews[0]
similarities = []

for review in reviews:
    if review['id'] != target_review['id']:
        sim = calculate_similarity(target_review['embedding'], review['embedding'])
        similarities.append((review, sim))

similarities.sort(key=lambda x: x[1], reverse=True)

print(f"Target review: '{target_review['text']}'\n")
print("Most similar reviews (potential duplicates):")
for review, sim in similarities[:3]:
    print(f"  [Similarity: {sim:.4f}] {review['text']}")

print("\n" + "="*80 + "\n")

# Application 3: Recommendation System
print("3️⃣ Recommendation System - 'Customers who liked this also liked...'")
print("="*80)

user_liked_review = reviews[2]  # User liked review #3
recommendations = semantic_search(user_liked_review['text'], reviews, top_k=4)
recommendations = [r for r in recommendations if r['id'] != user_liked_review['id']][:3]

print(f"User liked: '{user_liked_review['text']}'\n")
print("You might also like these products (based on similar reviews):")
for i, rec in enumerate(recommendations, 1):
    print(f"  {i}. {rec['text']}")

print("\n" + "="*80 + "\n")

# Application 4: Sentiment Analysis
print("4️⃣ Sentiment Analysis - Classify review sentiment")
print("="*80)

# Create reference embeddings for positive and negative
positive_ref = get_embedding("This product is amazing, high quality, and I love it!")
negative_ref = get_embedding("This product is terrible, poor quality, and I hate it!")

# Test on a new review
new_review = "The backpack is okay but the straps could be better"
new_embedding = get_embedding(new_review)

pos_sim = calculate_similarity(new_embedding, positive_ref)
neg_sim = calculate_similarity(new_embedding, negative_ref)

print(f"New review: '{new_review}'\n")
print(f"Similarity to positive reference: {pos_sim:.4f}")
print(f"Similarity to negative reference: {neg_sim:.4f}")
print(f"\nPredicted sentiment: {'Positive' if pos_sim > neg_sim else 'Negative'}")
print(f"Confidence: {abs(pos_sim - neg_sim):.4f}")

## 🎯 Key Takeaways

1. **Embeddings = Meaning as Numbers** - Text converted to vectors that capture semantic meaning

2. **Cosine Similarity Measures Relatedness** - Higher similarity = more similar meaning

3. **Semantic Search > Keyword Search** - Find content by meaning, not just matching words

4. **Similar Texts Cluster Together** - Embeddings naturally group related content

5. **Many Applications** - FAQ matching, duplicate detection, recommendations, sentiment analysis

6. **Visualization Helps Understanding** - PCA reduces dimensions for human interpretation

## 💰 Cost Considerations

Embedding costs (text-embedding-3-small):
- $0.02 per 1 million tokens
- Much cheaper than chat completions!
- Create once, reuse many times

## 💡 Best Practices

1. **Cache embeddings**: Don't recreate them every time
2. **Batch processing**: Embed multiple texts in one API call
3. **Choose the right model**: text-embedding-3-small vs text-embedding-3-large
4. **Normalize your data**: Clean text before embedding
5. **Store efficiently**: Use vector databases for large-scale applications

## 🚀 Try It Yourself!

Experiments to run:
1. Add your own reviews and see where they cluster
2. Try different search queries and examine the results
3. Experiment with different numbers of clusters (K-means)
4. Build a product recommendation system for your domain
5. Compare keyword search vs semantic search accuracy